## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime
import numpy as np



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [89]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid, name, check_sheet, ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    dict3 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                    if name in list(check_sheet.Workflow):
                        print(name)
                        dfy = check_sheet[check_sheet['Workflow']== name]

                        dfy.reset_index(drop=True)

                        for index,curie_id in enumerate(dfy.Curie):
                            print(index,curie_id)
                            node_num = dfy.iloc[index][3]
                            query_id = curie_id
                            if np.isnan(dfy.iloc[index][2]) == True:
                                len_check = len(child_response['fields']['data']['message']['results'])
                            else:
                                len_check = dfy.iloc[index][2]

                            #print(node_num, query_id, len_check)

                            locs = []
                            for x, val in enumerate(child_response['fields']['data']['message']['results']):
                                #print(val)

                                if x < len_check:

                                    if query_id in val['node_bindings'][node_num][0]['id']:
                                        locs.append(x)
                            if not locs:
                                check_result = f'False'
                                print(check_result)
                                #pass
                            else:
                                check_result = f'True'
                                print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                            dict3[curie_id] = check_result    

                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code) + ' ' + str(dict3)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [90]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [91]:
check_sheet

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
0,A.0_RHOBTB2_direct.json,PUBCHEM.COMPOUND:2950270,10.0,n1,Include,NaN,Note: this example row means that for workflow...
1,B.1_DILI-three-hop-interacts-with.json,CHEBI:41879,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
2,B.1_DILI-three-hop-interacts-with.json,MESH:D000077185,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
3,B.1_DILI-three-hop-interacts-with.json,MESH:D000077385,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
4,B.1_DILI-three-hop-interacts-with.json,MESH:D003474,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
5,B.1_DILI-three-hop-interacts-with.json,RXNORM:40068,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
6,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5877,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
7,B.1_DILI-three-hop-interacts-with.json,PUBCHEM.COMPOUND:5755,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
8,D.1_parkinsons-crohns.json,NCBIGene:120892,NaN,n01,include,NaN,LRRK2
9,D.1_parkinsons-crohns.json,NCBIGene:11315,NaN,n01,include,NaN,PARK7


<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [6]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            if (name == 'C.1_Template_SmallMolecule_real_world_evidence_Disease.json') or (name == 'C.2_Template_Drug_Disease_GeneSet_interacts_with_SmallMolecule.json') or (name == 'C.3_Template_Disease_related_to_Drug.json'):
                pass
            else:
                file_read = path.join(root, name)
                dir_name = (os.path.splitext(os.path.basename(root))[0])
                print(file_read)

                filename = (os.path.splitext(os.path.basename(file_read))[0])
                print(filename)
                with open(file_read,'r') as inf:
                    query = json.load(inf)

                    kcresult = submit_to_ars(query)

                    sleep(500)

                    result_status = retrieve_ars_results(kcresult, name, check_sheet)


                    dict_workflows[filename] = kcresult

                    sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=ed872b86-93d1-44d0-8022-6acfb0f774dd
Done
Done
kp-openpredict Done 0
Error
ara-robokop Error 0
Error
ara-aragorn-exp Error 0
Unknown
ara-ncats Unknown 0
Done
kp-molecular Done 99
Done
ara-unsecret Done 102
Done
ara-improving Done 5
Done
kp-chp Done 30
Error
ara-bte Error 0
Done
kp-textmining Done 0
Done
ara-explanatory Done 0
Done
kp-icees Done 0
Done
kp-icees-dili Done 0
Done
kp-genetics Done 0
Done
kp-cam Done 0
Done
ara-aragorn Done 951
Done
kp-cohd ARS Error 0
Done
ara-arax Done 163
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_RHOBTB2_twohop.json
A.2a_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=002b6841-2391-4e90-a882-d81a8dc41ea6
Done
Error
ara-robokop Error 0
Done
kp-textmining Done 0
Done
ara-unsecret Done 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Unknown
kp-genetics Unknown 0
Done
ara-aragorn Don

kp-cohd ARS Error 0
Done
kp-icees-dili Done 0
Done
ara-bte Done 305
Done
kp-genetics Done 0
Done
ara-aragorn Done 2562
Error
ara-unsecret Error 0
Error
ara-aragorn-exp Error 0
Done
kp-textmining Done 0
Done
kp-molecular Done 0
Done
ara-explanatory Done 0
Error
ara-robokop Error 0
Done
kp-icees Done 0
Done
ara-improving Done 16
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate.json
B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
https://arax.ncats.io/?source=ARS&id=48d353ac-1993-4678-89da-ad844d4b1d4d
Done
Done
kp-chp Done 0
Error
ara-arax Error 0
Error
ara-explanatory Error 0
Done
kp-textmining Done 0
Unknown
ara-ncats Unknown 0
Error
kp-icees-dili Error 0
Done
kp-cam Done 0
Done
ara-unsecret Done 0
Error
ara-aragorn Error 0
Error
kp-openpredict Error 0
Error
kp-molecular Error 0
Error
kp-cohd Error 0
Error
kp-genetics Error 0
Error
kp-icees Error 0
Error
ara-improving Error 0
Error
ara-robokop Erro

ara-bte Done 85
Done
ara-unsecret Done 0
Error
ara-aragorn Error 0
Done
kp-icees ARS Error 0
Done
ara-explanatory Done 0
Done
kp-icees-dili ARS Error 0
Done
kp-cam Done 0
Unknown
ara-ncats Unknown 0
Done
kp-textmining Done 0
Unknown
kp-genetics Unknown 0
Error
ara-improving Error 0
Done
kp-openpredict Done 0
Error
ara-robokop Error 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowC/C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule.json
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule
https://arax.ncats.io/?source=ARS&id=746177c2-1e65-41d0-9aab-66bae77cba66
Done
Done
ara-explanatory Done 0
Unknown
ara-arax Unknown 0
Unknown
kp-genetics Unknown 0
Error
kp-cohd Error 0
Done
ara-unsecret Done 0
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
ara-bte Done 0
Error
kp-molecular Error 0
Done
kp-chp Done 0
Done
kp-icees ARS Error 0
Error
ara-improving Error 0
Done
kp-cam Done 0
Error
ara-aragorn Error 0
Done
kp-icees-dili ARS Error 0
Unknown
ara-n

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [7]:
dict_workflows

{'A.3_KCNMA1': 'ed872b86-93d1-44d0-8022-6acfb0f774dd',
 'A.2a_RHOBTB2_twohop': '002b6841-2391-4e90-a882-d81a8dc41ea6',
 'A.8_EGFR_simple': '8f881db8-5a3d-4f5e-a4ff-83f29f61cf4d',
 'A.Multiomics_BigGIM_DR_KP_RHOBTB2': '702b93a6-557c-4905-9d26-1e332f122d83',
 'A.0_RHOBTB2_direct': '2cef056b-1f0e-433c-ad9f-764fee253e69',
 'A.2_RHOBTB2_twohop': '0d597bce-8cbf-48bf-a960-0446414549bd',
 'A.Multiomics_BigGIM_DR_KP_EGFR': '5b8d18af-a524-4eea-9de8-f7ffc6e82be6',
 'A.2_RHOBTB2_twohop_constrained': '6fa7a950-417c-4a40-bbc6-6461d87f9ed6',
 'A.9_EGFR_advanced': 'c3291e77-51e1-4cbf-9e98-4a054ce0b92f',
 'A.2a_expanded_RHOBTB2_twohop_constrained': 'e3a9776b-b8df-468b-9098-9b40e290aaae',
 'A.1_RHOBTB2': '15431437-7db1-4f07-ba2c-73191b03de47',
 'A.2a_expanded_RHOBTB2_twohop': '15573656-4cd2-4c89-b115-130be0b915ea',
 'B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol': '7cbc62af-a16e-41b3-a635-b8bf2ddf0eb6',
 'B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate': '48d353ac-1993-4678-89da-ad

In [92]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    name = keys + '.json'
    print(name, val)
    
    result_status = retrieve_ars_results(val, name, check_sheet)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1.json ed872b86-93d1-44d0-8022-6acfb0f774dd
Done
Done
kp-openpredict Done 0
Error
ara-robokop Error 0
Error
ara-aragorn-exp Error 0
Unknown
ara-ncats Unknown 0
Done
kp-molecular Done 99
Done
ara-unsecret Done 102
Done
ara-improving Done 5
Done
kp-chp Done 30
Error
ara-bte Error 0
Done
kp-textmining Done 0
Done
ara-explanatory Done 0
Done
kp-icees Done 0
Done
kp-icees-dili Done 0
Done
kp-genetics Done 0
Done
kp-cam Done 0
Done
ara-aragorn Done 951
Done
kp-cohd ARS Error 0
Done
ara-arax Done 163
A.2a_RHOBTB2_twohop.json 002b6841-2391-4e90-a882-d81a8dc41ea6
Done
Error
ara-robokop Error 0
Done
kp-textmining Done 0
Done
ara-unsecret Done 0
Done
kp-openpredict Done 0
Done
kp-icees ARS Error 0
Unknown
kp-genetics Unknown 0
Done
ara-aragorn Done 0
Error
kp-cohd Error 0
Done
ara-improving ARS Error 0
Error
kp-molecular Error 0
Done
kp-cam Done 0
Done
kp-chp Done 0
Done
ara-explanatory Done 0
Unknown
ara-ncats Unknown 0
Done
kp-icees-dili ARS Error 0
Error
ara-bte Error 0
Done
ara-arax 

kp-textmining Done 0
Done
kp-cam Done 0
Error
ara-bte Error 0
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone.json 7ef10ebe-ae95-446a-8ec2-0189a394759b
Done
Done
ara-improving Done 219
Done
kp-chp Done 0
Done
kp-molecular Done 242
Done
ara-unsecret Done 347
Error
ara-aragorn Error 0
Done
ara-bte Done 2817
Done
kp-icees-dili Done 0
Done
ara-arax Done 500
Done
kp-cam Done 0
Error
ara-robokop Error 0
Done
kp-textmining Done 0
Done
kp-cohd Done 50
Done
kp-openpredict Done 300
Done
kp-icees Done 0
Done
ara-explanatory Done 0
Done
kp-genetics Done 0
Error
ara-aragorn-exp Error 0
Unknown
ara-ncats Unknown 0
B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin.json 4c5c22ff-70ce-4879-9534-a6ee91756a1f
Done
Error
ara-aragorn-exp Error 0
Done
kp-textmining Done 0
Unknown
ara-ncats Unknown 0
Done
ara-arax Done 500
Done
kp-chp Done 0
Done
kp-icees-dili Done 0
Error
ara-robokop Error 0
Done
kp-molecular Done 0
Done
kp-openpredict Done 0
Done
kp-icees Done 0
Done
ara-explanatory Done 0
Error


ara-aragorn-exp Error 0
Done
kp-textmining Done 0
Done
ara-unsecret Done 546
Done
kp-molecular Done 546
C.3a_MultSclerosis_related_to_Nimodipine.json 76ccc6de-13bb-4bbd-9e52-71cf32333952
Done
Done
kp-icees Done 0
Done
kp-chp Done 0
Done
ara-improving Done 0
Done
kp-molecular Done 0
Error
ara-aragorn-exp Error 0
Done
ara-bte Done 0
Done
kp-textmining Done 0
Error
ara-robokop Error 0
Done
ara-arax Done 1
Done
kp-icees-dili Done 0
Error
ara-unsecret Error 0
Done
ara-explanatory Done 0
Error
ara-aragorn Error 0
Done
kp-cohd Done 0
Done
kp-genetics Done 0
Done
kp-openpredict Done 0
Unknown
ara-ncats Unknown 0
Done
kp-cam Done 0
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule.json f67cfce9-593f-4db6-a621-05836043aa3b
Done
Done
kp-openpredict Done 0
Error
ara-aragorn-exp Error 0
Done
ara-explanatory Done 0
Unknown
kp-genetics Unknown 0
Done
kp-chp Done 0
Done
kp-textmining Done 0
Done
ara-unsecret Done 0
Done
kp-cam Done 0
Error
ara-aragorn Error 0
Error
ara-improving Error 0
Done
kp-ic

In [93]:
workflow_result_messages

{'A.3_KCNMA1': [{'pk_id': 'https://arax.ncats.io/?source=ARS&id=ed872b86-93d1-44d0-8022-6acfb0f774dd',
   'kp-openpredict': 'No Results: 200',
   'ara-robokop': 'Error: 500',
   'ara-aragorn-exp': 'Error: 404',
   'ara-ncats': 'Unknown: 503',
   'kp-molecular': 'Results: 200 {}',
   'ara-unsecret': 'Results: 200 {}',
   'ara-improving': 'Results: 200 {}',
   'kp-chp': 'Results: 200 {}',
   'ara-bte': 'Error: 598',
   'kp-textmining': 'No Results: 200',
   'ara-explanatory': 'No Results: 200',
   'kp-icees': 'No Results: 200',
   'kp-icees-dili': 'No Results: 200',
   'kp-genetics': 'No Results: 200',
   'kp-cam': 'No Results: 200',
   'ara-aragorn': 'Results: 200 {}',
   'kp-cohd': 'ARS Error: 200',
   'ara-arax': 'Results: 200 {}'},
  {'kp-molecular': {'infores:chembl',
    'infores:ctd',
    'infores:dgidb',
    'infores:drugbank',
    'infores:gtopdb',
    'infores:hmdb',
    'infores:molepro'},
   'ara-unsecret': {'infores:chembl',
    'infores:ctd',
    'infores:dgidb',
    'infor

### Creating dataframe for workflows with PK

<br>

In [103]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)


for k in sorted(workflow_result_messages):
    print(k)
    
    col.append(k)
    
    
    
    #count = 0
    for key, value in workflow_result_messages[k][0].items():
        
        
        #count= count+1
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    #print(count)
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-interacts-with
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.2f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
B.3_DILI_branched-four-hop
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule
C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule
C.3a_MultSclerosis_related_t

In [104]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=2cef056b-...,https://arax.ncats.io/?source=ARS&id=15431437-...,https://arax.ncats.io/?source=ARS&id=0d597bce-...,https://arax.ncats.io/?source=ARS&id=6fa7a950-...,https://arax.ncats.io/?source=ARS&id=002b6841-...,https://arax.ncats.io/?source=ARS&id=15573656-...,https://arax.ncats.io/?source=ARS&id=e3a9776b-...,https://arax.ncats.io/?source=ARS&id=ed872b86-...,https://arax.ncats.io/?source=ARS&id=8f881db8-...,https://arax.ncats.io/?source=ARS&id=c3291e77-...,...,https://arax.ncats.io/?source=ARS&id=99b283fe-...,https://arax.ncats.io/?source=ARS&id=f67cfce9-...,https://arax.ncats.io/?source=ARS&id=746177c2-...,https://arax.ncats.io/?source=ARS&id=22e4ae6f-...,https://arax.ncats.io/?source=ARS&id=76ccc6de-...,https://arax.ncats.io/?source=ARS&id=b3a39c22-...,https://arax.ncats.io/?source=ARS&id=58ef1e94-...,https://arax.ncats.io/?source=ARS&id=47d93f67-...,https://arax.ncats.io/?source=ARS&id=006f6ba2-...,https://arax.ncats.io/?source=ARS&id=def0f55e-...
kp-icees-dili,No Results: 200,No Results: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,No Results: 200,ARS Error: 200,ARS Error: 200,No Results: 200,ARS Error: 200,ARS Error: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
kp-molecular,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,No Results: 200,Error: 501,Error: 501,Error: 501,Error: 501,Error: 501,Results: 200 {},Results: 200 {},Results: 200 {},...,Results: 200 {},Error: 501,Error: 501,Error: 501,No Results: 200,Error: 501,Error: 501,No Results: 200,Error: 501,Error: 501
kp-genetics,No Results: 200,No Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,No Results: 200,Unknown: 503,Unknown: 503,No Results: 200,Unknown: 503,Unknown: 503
kp-icees,No Results: 200,No Results: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,ARS Error: 200,ARS Error: 200,ARS Error: 200,No Results: 200,ARS Error: 200,ARS Error: 200,No Results: 200,ARS Error: 200,ARS Error: 200
kp-openpredict,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-bte,No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Error: 598,Results: 200 {},Results: 200 {},Error: 598,Error: 598,Results: 200 {},...,Error: 598,Error: 598,No Results: 200,Results: 200 {},No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 598
ara-aragorn,No Results: 200,Results: 200 {},Error: 598,No Results: 200,No Results: 200,Results: 200 {},No Results: 200,Results: 200 {},Results: 200 {},No Results: 200,.

In [105]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [106]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=2cef056b-...,https://arax.ncats.io/?source=ARS&id=15431437-...,https://arax.ncats.io/?source=ARS&id=0d597bce-...,https://arax.ncats.io/?source=ARS&id=6fa7a950-...,https://arax.ncats.io/?source=ARS&id=002b6841-...,https://arax.ncats.io/?source=ARS&id=15573656-...,https://arax.ncats.io/?source=ARS&id=e3a9776b-...,https://arax.ncats.io/?source=ARS&id=ed872b86-...,https://arax.ncats.io/?source=ARS&id=8f881db8-...,https://arax.ncats.io/?source=ARS&id=c3291e77-...,...,https://arax.ncats.io/?source=ARS&id=99b283fe-...,https://arax.ncats.io/?source=ARS&id=f67cfce9-...,https://arax.ncats.io/?source=ARS&id=746177c2-...,https://arax.ncats.io/?source=ARS&id=22e4ae6f-...,https://arax.ncats.io/?source=ARS&id=76ccc6de-...,https://arax.ncats.io/?source=ARS&id=b3a39c22-...,https://arax.ncats.io/?source=ARS&id=58ef1e94-...,https://arax.ncats.io/?source=ARS&id=47d93f67-...,https://arax.ncats.io/?source=ARS&id=006f6ba2-...,https://arax.ncats.io/?source=ARS&id=def0f55e-...
kp-icees-dili,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
kp-molecular,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,No Results: 200,Error: 501,Error: 501,Error: 501,Error: 501,Error: 501,Results: 200 {},Results: 200 {},Results: 200 {},...,Results: 200 {},Error: 501,Error: 501,Error: 501,No Results: 200,Error: 501,Error: 501,No Results: 200,Error: 501,Error: 501
kp-genetics,No Results: 200,No Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,No Results: 200,Unknown: 503,Unknown: 503,No Results: 200,Unknown: 503,Unknown: 503
kp-icees,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
kp-openpredict,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-bte,No Results: 200,Results: 200 {},Results: 200 {},Results: 200 {},Error: 598,Results: 200 {},Results: 200 {},Error: 598,Error: 598,Results: 200 {},...,Error: 598,Error: 598,No Results: 200,Results: 200 {},No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 598
ara-aragorn,No Results: 200,Results: 200 {},Error: 598,No Results: 200,No Results: 200,Results: 200 {},No Results: 200,Results: 200 {},Results: 

In [107]:
df.replace('{}','',regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [17]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-interacts-with
B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.2f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone
B.3_DILI_branched-four-hop
C.1a_SmallMolecule_real_world_evidence_MultSclerosis
C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule
C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule
C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule
C.3a_MultSclerosis_related_t

In [18]:
final_dict2

defaultdict(dict,
            {'A.0_RHOBTB2_direct': {'kp-molecular': {'infores:ctd',
               'infores:molepro'},
              'ara-arax': {'infores:arax', 'infores:ctd', 'infores:molepro'},
              'ara-explanatory': {'infores:automat-ctd',
               'infores:automat-robokop',
               'infores:biothings-multiomics-biggim-drugresponse',
               'infores:ctd',
               'infores:explanatory-agent',
               'infores:lincs',
               'infores:molepro',
               'infores:spoke',
               'infores:translator-biothings-explorer'},
              'ara-unsecret': {'infores:ctd', 'infores:molepro'},
              'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.1_RHOBTB2': {'ara-explanatory': {'infores:explanatory-agent'},
              'ara-arax': {'infores:arax', 'infores:rtx-kg2'},
              'ara-bte': {'infores:biothings-semmeddb-gene',
            

In [19]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [20]:
final_dictassemble

[['A.0_RHOBTB2_direct', 'kp-molecular', 'infores:molepro'],
 ['A.0_RHOBTB2_direct', 'kp-molecular', 'infores:ctd'],
 ['A.0_RHOBTB2_direct', 'ara-arax', 'infores:molepro'],
 ['A.0_RHOBTB2_direct', 'ara-arax', 'infores:arax'],
 ['A.0_RHOBTB2_direct', 'ara-arax', 'infores:ctd'],
 ['A.0_RHOBTB2_direct',
  'ara-explanatory',
  'infores:translator-biothings-explorer'],
 ['A.0_RHOBTB2_direct', 'ara-explanatory', 'infores:automat-ctd'],
 ['A.0_RHOBTB2_direct', 'ara-explanatory', 'infores:ctd'],
 ['A.0_RHOBTB2_direct',
  'ara-explanatory',
  'infores:biothings-multiomics-biggim-drugresponse'],
 ['A.0_RHOBTB2_direct', 'ara-explanatory', 'infores:automat-robokop'],
 ['A.0_RHOBTB2_direct', 'ara-explanatory', 'infores:molepro'],
 ['A.0_RHOBTB2_direct', 'ara-explanatory', 'infores:lincs'],
 ['A.0_RHOBTB2_direct', 'ara-explanatory', 'infores:explanatory-agent'],
 ['A.0_RHOBTB2_direct', 'ara-explanatory', 'infores:spoke'],
 ['A.0_RHOBTB2_direct', 'ara-unsecret', 'infores:molepro'],
 ['A.0_RHOBTB2_dire

In [21]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [22]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [23]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [24]:
df2test = pd.DataFrame(df2test.unstack().T)

In [25]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,NaN,NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,NaN,NaN
icees-asthma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[kp-icees],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:aragorn,NaN,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:aragorn-ranker-ara,NaN,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:arax,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,[ara-arax],NaN,[ara-arax],NaN,[ara-arax],NaN,NaN,[ara-arax],NaN,NaN
infores:automat-biolink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-covidkop,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN,[ara-aragorn],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-ctd,[ara-explanatory],NaN,[ara-bte],[ara-bte],"[ara-aragorn, ara-bte]",[ara-bte],[ara-aragorn],[ara-aragorn],"[ara-arax, ara-bte]","[ara-arax, ara-aragorn, ara-bte]",...,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#df2test.drop([''], axis=0, inplace=True)

In [27]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [28]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
Values,,,,,,,,,,,,,,,,,,,,,
CTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,NaN,NaN
DrugCentral Indication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,NaN,NaN
icees-asthma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[kp-icees],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:aragorn,NaN,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:aragorn-ranker-ara,NaN,[ara-aragorn],NaN,NaN,[ara-aragorn],NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:arax,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,[ara-arax],NaN,[ara-arax],NaN,[ara-arax],NaN,NaN,[ara-arax],NaN,NaN
infores:automat-biolink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-covidkop,NaN,NaN,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN,[ara-aragorn],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-ctd,[ara-explanatory],NaN,[ara-bte],[ara-bte],"[ara-aragorn, ara-bte]",[ara-bte],[ara-aragorn],[ara-aragorn],"[ara-arax, ara-bte]","[ara-arax, ara-aragorn, ara-bte]",...,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#df2.replace([], 'None', regex=True,inplace=True)

In [30]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [31]:
df2test = df2test.rename_axis(None)

In [32]:
df2test.columns.name = None

In [33]:
df2test.fillna('', inplace=True)

In [34]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
CTD,,,,,,,,,,,...,,,,[ara-bte],,,,,,
DrugCentral Indication,,,,,,,,,,,...,,,,[ara-bte],,,,,,
icees-asthma,,,,,,,,,,,...,[kp-icees],,,,,,,,,
infores:aragorn,,[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,,,,,,,,
infores:aragorn-ranker-ara,,[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,,,,,,,,
infores:arax,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,[ara-arax],,[ara-arax],,[ara-arax],,,[ara-arax],,
infores:automat-biolink,,,,,,,,,,,...,,,,,,,,,,
infores:automat-covidkop,,,,,[ara-aragorn],,,,,[ara-aragorn],...,,,,,,,,,,
infores:automat-ctd,[ara-explanatory],,[ara-bte],[ara-bte],"[ara-aragorn, ara-bte]",[ara-bte],[ara-aragorn],[ara-aragorn],"[ara-arax, ara-bte]","[ara-arax, ara-aragorn, ara-bte]",...,,,,[ara-bte],,,,,,
infores:automat-drug-central,,,,[ara-arax],,,[ara-aragorn],[ara-aragorn],,,...,,,,,,,,,,


## Sorting The index to push infores ID's that are compliant to Column L and M in INFORES Catalog


<br>
<br>

In [35]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [36]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]

In [37]:
infores_catalog = infores_catalog[:335]

In [38]:
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

In [39]:
dict_map

{'CTD': 'Illegal value',
 'DrugCentral Indication': 'Illegal value',
 'icees-asthma': 'Illegal value',
 'infores:aragorn': 'ARA',
 'infores:aragorn-ranker-ara': 'Illegal value',
 'infores:arax': 'ARA',
 'infores:automat-biolink': 'KP',
 'infores:automat-covidkop': 'KP',
 'infores:automat-ctd': 'KP',
 'infores:automat-drug-central': 'KP',
 'infores:automat-gtopdb': 'KP',
 'infores:automat-hetio': 'KP',
 'infores:automat-hmdb': 'KP',
 'infores:automat-pharos': 'KP',
 'infores:automat-robokop': 'KP',
 'infores:bindingdb': 'External Source',
 'infores:biothings-dgidb': 'KP',
 'infores:biothings-multiomics-biggim-drugresponse': 'KP',
 'infores:biothings-semmeddb-gene': 'KP',
 'infores:chembl': 'External Source',
 'infores:civic': 'External Source',
 'infores:cohd': 'KP',
 'infores:connections-hypothesis': 'KP',
 'infores:cord19-scibite': 'External Source',
 'infores:ctd': 'External Source',
 'infores:dgidb': 'External Source',
 'infores:diseases': 'External Source',
 'infores:disgenet': 'Ex

In [40]:
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)

In [41]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog
CTD,,,,,,,,,,,...,,,[ara-bte],,,,,,,Illegal value
DrugCentral Indication,,,,,,,,,,,...,,,[ara-bte],,,,,,,Illegal value
icees-asthma,,,,,,,,,,,...,,,,,,,,,,Illegal value
infores:aragorn,,[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,,,,,,,,ARA
infores:aragorn-ranker-ara,,[ara-aragorn],,,[ara-aragorn],,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,,,,,,,,Illegal value
infores:arax,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,,[ara-arax],,[ara-arax],,,[ara-arax],,,ARA
infores:automat-biolink,,,,,,,,,,,...,,,,,,,,,,KP
infores:automat-covidkop,,,,,[ara-aragorn],,,,,[ara-aragorn],...,,,,,,,,,,KP
infores:automat-ctd,[ara-explanatory],,[ara-bte],[ara-bte],"[ara-aragorn, ara-bte]",[ara-bte],[ara-aragorn],[ara-aragorn],"[ara-arax, ara-bte]","[ara-arax, ara-aragorn, ara-bte]",...,,,[ara-bte],,,,,,,KP
infores:automat-drug-central,,,,[ara-arax],,,[ara-aragorn],[ara-aragorn],,,...,,,,,,,,,,KP


In [42]:
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)

In [43]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin,Translator_Category_Complaint_to_ColL&M_InforesCatalog,sort
infores:biothings-dgidb,,,[ara-bte],[ara-bte],,,,,,,...,,[ara-bte],,,,,,,KP,KP
infores:lincs,"[ara-explanatory, ara-improving]","[ara-improving, ara-aragorn]","[ara-bte, ara-improving]","[ara-improving, ara-bte]",[ara-aragorn],,"[ara-improving, ara-aragorn]",[ara-improving],[ara-improving],,...,,,,,[ara-improving],,,,KP,KP
infores:molepro,"[kp-molecular, ara-arax, ara-explanatory, ara-...",,[ara-arax],[ara-arax],[ara-aragorn],,"[kp-molecular, ara-unsecret, ara-aragorn, ara-...","[ara-aragorn, ara-arax, kp-molecular, ara-unse...","[kp-molecular, ara-arax, ara-unsecret]","[ara-aragorn, ara-unsecret, kp-molecular]",...,,,,,,,,,KP,KP
infores:mychem-info,,,,[ara-arax],,,,,,,...,,,,,,,,,KP,KP
infores:mydisease-info,,,,,,,,,,,...,,[ara-bte],,,,,,,KP,KP
infores:openpredict,,,,,,,,,,,...,,,,,,,,,KP,KP
infores:connections-hypothesis,,,[ara-arax],[ara-arax],,,"[kp-chp, ara-aragorn, ara-arax]",,,,...,,,,,,,,,KP,KP
infores:cohd,,,,,,,,,,,...,,,,,,[kp-cohd],,,KP,KP
infores:rtx-kg2,,"[ara-arax, ara-aragorn]",[ara-arax],[ara-arax],"[ara-aragorn, ara-arax]",[ara-arax],"[ara-aragorn, ara-arax]","[ara-aragorn, ara-arax]",[ara-arax],,...,[ara-arax],,[ara-arax],,,[ara-arax],,,KP,KP
infores:spoke,"[ara-explanatory, ara-improving]","[ara-improving, ara-aragorn]",[ara-improving],[ara-improving],[ara-aragorn],,[ara-improving],[ara-improving],[ara-improving],,...,,,,[ara-improving],[ara-improving],[ara-improving],[ara-improving],[ara-improving],KP,KP


In [44]:
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]

In [45]:
df2test['Query Type'] = 'Sync'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

<br>

## Converting the Pk's to hyperlink

<br>

In [108]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [109]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [110]:
df.sort_index(inplace=True)

In [111]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,...,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Unknown: 503,Results: 200,Unknown: 503,Unknown: 503,Results: 200,Error: 598,Unknown: 503,No Results: 200,Unknown: 503,Error: 502
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200,...,Error: 598,Error: 598,No Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 598
ara-explanatory,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'True'},Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,No Results: 200,Error: 400,Error: 400,Error: 400,No Results: 200,"Results: 200 {'NCBIGene:120892': 'True', 'NCBI...","Results: 200 {'NCBIGene:3988': 'True', 'NCBIGe...",No Results: 200,Results: 200,No Results: 200
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-unsecret,Results: 200 {'PUBCHEM.COMPOUND:2950270': 'Fal...,No Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,No Results: 200,No Results: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200


<br>

### Highlight the cells In Excel

<br>

In [50]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [51]:
styled = df.style.applymap(highlight)

In [52]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.1_DILI-three-hop-interacts-with,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.2f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.3_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 500,Error: 598,Results: 200,Results: 200,Results: 200,Error: 422,Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 400,Results: 200,Results: 200,Error: 598,Unknown: 503,Results: 200,Unknown: 503,Unknown: 503,Results: 200,Error: 598,Unknown: 503,Results: 200,Unknown: 503,Error: 502
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200,Results: 200,Error: 502,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,No Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 598
ara-explanatory,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,No Results: 200,No Results: 200,Error: 400,Error: 400,Error: 400,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503

In [53]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [99]:
wks_name

'Workflow Progress Tracker_2021_09_29-08_27_23_AM'

In [55]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

/usr/local/lib/python3.9/site-packages/pandas/io/formats/excel.py:375: CSSWarning: Unhandled color format: 'magneta'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [56]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.1_DILI-three-hop-interacts-with,B.2a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.2b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.2c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.2d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.2e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.2f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.2g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5865_Prednisone,B.3_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule,C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule,C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 500,Error: 598,Results: 200,Results: 200,Results: 200,Error: 422,Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 400,Results: 200,Results: 200,Error: 598,Unknown: 503,Results: 200,Unknown: 503,Unknown: 503,Results: 200,Error: 598,Unknown: 503,Results: 200,Unknown: 503,Error: 502
ara-bte,No Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Results: 200,Error: 598,Error: 598,Results: 200,Results: 200,Error: 502,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,Error: 598,No Results: 200,Results: 200,No Results: 200,Error: 598,Error: 598,Error: 598,No Results: 200,Error: 598
ara-explanatory,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Error: 400,Results: 200,Results: 200,No Results: 200,No Results: 200,Error: 400,Error: 400,Error: 400,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503

In [115]:
df['Query Type'] = 'Sync'

df = df[['Query Type']+ list(df.columns[:-1])]

In [58]:
#df.reset_index(inplace=True)

In [59]:
#df.rename({'index': 'ARA/KP'}, axis=1, inplace=True)

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [117]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [118]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [119]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Workflow Progress Tracker_2021_09_29-08_27_23_AM' id:1862962344>

### Push Dataframe 2

<br>

In [64]:
wks2 = 'edge_attribute_source_' + date

In [65]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2test, spreadsheet_key, wks2, credentials=credentials, row_names=True)

<Worksheet 'edge_attribute_source_2021_09_29-08_27_23_AM' id:916120098>